In [10]:
import time
import glob
import nltk
import json
import string
import numpy as np
import pandas as pd
import os
import natsort
import math
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from collections import Counter, OrderedDict
from timeit import default_timer as timer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


In [11]:
text_dictionary = []
for i in range(len(os.listdir('D:/TH_TVDPT/do_an/Cranfield/'))):
    idx = i + 1
    information_doc = []
    file = 'D:/TH_TVDPT/do_an/Cranfield/' + str(idx) + '.txt'
    with open(file) as f:
        info = f.read()
        info = re.sub('[@*!#$^&)(_+=-?.,/]',"",info)
        information_doc.append(info)
    text_dictionary.append(information_doc)

#print('Text dictionary: ', len(text_dictionary), type(text_dictionary))

#print(text_dictionary)

In [12]:
os.chdir("D:\TH_TVDPT\do_an")
Result_List = {} #chứa các file txt liên quan tới từng câu truy vấn


In [13]:
filePath = "D:\TH_TVDPT\do_an\TEST\RES"
Cranfield = os.listdir(filePath)
Cranfield= natsort.natsorted(Cranfield)
Cranfield_List = []
''' Truy cập kết quả Cranfield'''
for f in Cranfield:
    r = open(filePath+ "\\" + f,"r",encoding="utf-8")
    column_1 = []
    for row in r:
        txt = row.split()
        txt = txt[1]+".txt"
        column_1.append(txt)
    Cranfield_List.append(column_1)
#print(Cranfield_List)

# Xử lý tài liệu Cranfield

In [14]:
# Create list word non stop words
start_time = time.time()
stop_words = stopwords.words('english') + list(string.punctuation) + ['\n']
Word_list = []
i = 1
for text in text_dictionary:
    for word in word_tokenize(text[0].lower().strip()):
        #if word not in stop_words:
        a = []
        a.append(word)
        a.append(i)
        Word_list.append(a)
    i += 1
#print('Word list: ', len(Word_list), type(Word_list))
#print(Word_list)

In [15]:
'''
#stemming bằng lemmatizer
lemmatizer = WordNetLemmatizer()
lemmaList= Word_list
i = 0
for word in Word_list:
    #lemmaList.append(lemmatizer.lemmatize(word))
    #print(word[0])
    lemmaList[i][0] = lemmatizer.lemmatize(word[0])
    i += 1
#print(lemmaList)
'''

'\n#stemming bằng lemmatizer\nlemmatizer = WordNetLemmatizer()\nlemmaList= Word_list\ni = 0\nfor word in Word_list:\n    #lemmaList.append(lemmatizer.lemmatize(word))\n    #print(word[0])\n    lemmaList[i][0] = lemmatizer.lemmatize(word[0])\n    i += 1\n#print(lemmaList)\n'

In [16]:
#stemming bằng porterStemmer
start_time = time.time()
ps = PorterStemmer()
ps_list = Word_list
i = 0

for word in Word_list:

    #ps_list.append(ps.stem(word))
    ps_list[i][0] = ps.stem(word[0])
    i += 1
#ps_list_set = []
#ps_list_set = set(ps_list)
#print('Ps_list: ', len(ps_list_set), type(ps_list_set))
#print(ps_list_set)
#print(ps_list)


In [19]:
print(len(ps_list))

225470


In [17]:
#Tính ra từ trong dic xuất hiện bao nhiêu lần trong 1 tài liệu ( posting list chưa tối ưu)
create = []
for i in ps_list:
    create.append(i[0])
create = set(create)
ps_list_set = []
for i in create:
    a = []
    a.append(i)
    ps_list_set.append(a)
#print(ps_list_set)
for i in ps_list_set:
    #print(type(i[0]))
    b = []
    for x in ps_list:
        if i[0] in x:
            b.append(x[1])
    i.append(b)
print("--- %s seconds ---" % (time.time() - start_time))
print(len(ps_list_set))
#print(ps_list_set)# Lưu file Dictionary & PostingList

KeyboardInterrupt: 

# Lưu file Dictionary & PostingList

In [9]:
# Write file 
if os.path.exists('TF_dictionary_Non_Stop_Stemming.json') == True:
  os.remove('TF_dictionary_Non_Stop_Stemming.json')
  print('Remove old TF_dictionary.json file')
with open('TF_dictionary_Non_Stop_Stemming.json', 'a+') as f:
  json.dump(ps_list_set, f)
  print('Create new TF_dictionary.json file')# Load File

Remove old TF_dictionary.json file
Create new TF_dictionary.json file


# Load File

In [121]:
with open('TF_dictionary_Non_Stop_Stemming.json', 'r') as f:
    ps_list_set = json.load(f) 

In [122]:
Dictionary = {}
for i in ps_list_set:
    posting_list = []
    for x in set(i[1]):
        a = i[1].count(x)
        b = [x,a]
        posting_list.append(b)
    Dictionary[i[0]] =(len(set(i[1])),posting_list)

In [123]:
#print(Dictionary)

In [124]:
#Tính tf_idf
Dictionary_new = {}
idf_dict = {}
for key in Dictionary:
    #idf = 1/Dictionary[key][0]
    #idf = 1400/Dictionary[key][0]
    #idf = math.log((1400/Dictionary[key][0]) + 1)
    #idf = abs(math.log(1/Dictionary[key][0]))
    idf = math.log(1400/Dictionary[key][0])
    a = []
    for i in Dictionary[key][1][:][:]:
        #print(i[1])
        b = []
        tf_idf = i[1]*idf
        b = [i[0],tf_idf]
        a.append(b)
    Dictionary_new[key] =(Dictionary[key][0],idf,a)
    idf_dict[key] = (idf)
    #print(Dictionary[key][1][:][:][1])
    #for i in Dictionary[key][1][:][:]:
    #    print(i[])

In [125]:
#print(Dictionary_new)

In [126]:
#Lấy danh sách posting có chứa tf_idf
posting = []
for key in Dictionary_new:
    for i in Dictionary_new[key][2][:][:]:
        posting.append(i)


In [127]:
#Gom tf_idf của mỗi tìa liệu
norm_dict = {}
for i in posting:
    if norm_dict.get(i[0]) is None:
        norm_dict[i[0]] = ([(i[1])])
    else:
        norm_dict[i[0]].append(i[1])

In [128]:
#print(norm_dict)

In [129]:
#TÍnh norm của mỗi tài liệu
norm_dict_1 = {}
for key in norm_dict:
    list_norm = norm_dict[key]
    norm = 0
    sum_ = 0
    for i in list_norm:
        sum_ += i**2
    norm = math.sqrt(sum_)
    norm_dict_1[key]=(norm)
  

In [130]:
#print(norm_dict_1)

In [131]:
result_Dictionary = {}
for key in Dictionary_new:
    a =[]
    for i in Dictionary_new[key][2][:][:]:
        #print(i[1])
        chia = i[1]/norm_dict_1[i[0]]
        b = []
        b.append(i[0])
        b.append(chia)
        a.append(b)
    result_Dictionary[key] = (Dictionary_new[key][0],Dictionary_new[key][1],a)

In [132]:
#print(result_Dictionary)

# Xử lý câu truy vấn

In [133]:
''' Truy cập file câu hỏi'''
querry_source = open("D:/TH_TVDPT/do_an/TEST/query.txt","r",encoding="utf-8")
index = 1
querry_dict = {}
for querry in querry_source:
    qr_split = querry.split("\t")
    
    querry_dict[index] = (qr_split[1].replace('\n',''))
    index +=1 
    
#for  in range(len(Result_List)):
#    Result_List[querry] = Result_List[querry].replace('\n','')
#print(querry_dict)

In [134]:

Word_list_querry_stemming_Normal_Cosin = {} # tính cosin
result_id = 1
for key  in querry_dict:
    
    Word_list_querry = {}
    Word_list_querry_stemming = {} # stemming và stop word
    Norm_dict_querry = {} # tính norm
    Word_list_querry_stemming_Normal ={} # chuẩn hóa

    stop_words = stopwords.words('english') + list(string.punctuation) + ['\n']
    
    for word in word_tokenize(querry_dict[key].lower().strip()):
        #if word not in stop_words:
        word = ps.stem(word)
        if Word_list_querry.get(word) is None:
            Word_list_querry[word] = (1)
        else:
            Word_list_querry[word] = (int(Word_list_querry[word])+1)
    #print('Word list: ', len(Word_list_querry), type(Word_list_querry))
    #print(Word_list_querry)
    ###############################################################################

    
    for key_1 in Word_list_querry:
        if idf_dict.get(key_1) != None:
            Word_list_querry_stemming[key_1] = (Word_list_querry[key_1]*idf_dict[key_1])
        else:
            Word_list_querry_stemming[key_1] = (0)
            
    #print(Word_list_querry_stemming)
    ##################################################################################
    #Chuẩn hóa
    tong = 0
    for key_1 in Word_list_querry_stemming:
        tong = tong + (Word_list_querry_stemming[key_1]**2)
    tong = math.sqrt(tong)
    for key_1 in Word_list_querry_stemming:
        Word_list_querry_stemming_Normal[key_1] = Word_list_querry_stemming[key_1]/tong
    #print(Word_list_querry_stemming_Normal)
    ##################################################################################
    #Độ đo tương đồng
    for key_1 in Word_list_querry_stemming_Normal:
        if result_Dictionary.get(key_1) != None:
            a =[]
            for i in result_Dictionary[key_1][2][:][:]:
                #print(i[1])
                nhan = Word_list_querry_stemming_Normal[key_1]*i[1]
                b = []
                b.append(i[0])
                b.append(nhan)
                a.append(b)
            a.sort(key=lambda x: x[1], reverse=True)
            Word_list_querry_stemming_Normal_Cosin[result_id] = (a)
        #else:
        #    Word_list_querry_stemming_Normal_Cosin[result_id] = ([0,0])
        
    result_id += 1
    #print(Word_list_querry_stemming_Normal_Cosin)

In [135]:
#print(Word_list_querry_stemming_Normal_Cosin)

In [136]:
result_querry_lates = []
for key in Word_list_querry_stemming_Normal_Cosin:
    result_querry = {}
    a = []
    for i in Word_list_querry_stemming_Normal_Cosin[key]:
        #print(i[0])
        if result_querry.get(i[0]) != None:
            result_querry[i[0]] = (float(result_querry[i[0]]) + float(i[1]))
        else:
            result_querry[i[0]] = (float(i[1]))
    for key in result_querry:
        a.append(key)
    result_querry_lates.append(a)
    #print(result_querry)

In [137]:
#print(result_querry_lates)

# Tính AP ,MAP

In [138]:
filePath = "D:\TH_TVDPT\do_an\TEST\RES"
Cranfield = os.listdir(filePath)
Cranfield= natsort.natsorted(Cranfield)
Cranfield_List = []
''' Truy cập kết quả Cranfield'''
for f in Cranfield:
    r = open(filePath+ "\\" + f,"r",encoding="utf-8")
    column_1 = []
    for row in r:
        txt = row.split()
        txt = txt[1]
        column_1.append(int(txt))
    Cranfield_List.append(column_1)
#print(Cranfield_List)

In [139]:
#print(type(result_querry_lates))
#print(type(Cranfield_List))

Precision = []
Recall = []
for index in range(len(result_querry_lates)):
    True_False = []
    for cau in result_querry_lates[index]:
        if cau in Cranfield_List[index]:
            True_False.append("True")
        else:
            True_False.append("False")
    Result_True = len(Cranfield_List[index])
    n = 0 # Tổng số kết quả trả về
    y = 1 # Tổng số kết quả trả về đúng
    x = 0 # 
    p = []
    r = []
    for i in True_False:
        n += 1

        if str(i) == "True":
            p.append(y/n)
            r.append(y/Result_True)
            y += 1
    Precision.append(p)
    Recall.append(r)

In [140]:
Average_Precision = []
percent = []
#print(Precision[0].index(max(Precision[0])))


for i in range(len(Precision)):
    if(len(Precision[i]) != 0):
        percent1 = 0.1 #phan tram do phu
        #percent2 = 0.1 #phan tram do phu
        stt = 0
        percent11 = []
        percent11.append(max(Precision[i]))
        while(True):
            max_p = max(Precision[i][stt:])
            if Recall[i][stt] >= percent1:
                percent11.append(max_p)
                percent1 += 0.1
            stt += 1
            if stt == len(Precision[i]):
                if len(percent11) != 11:
                    percent11.append(0)
                break
        percent.append(percent11)
for i in range(len(percent)):
    if(str(np.average(percent[i])) != "nan"):
        Average_Precision.append(np.average(percent[i]))
    else:
        Average_Precision.append(float(0))


In [141]:
print("mean Average Precision:", np.average(Average_Precision))

mean Average Precision: 0.15290480246728094
